Übung: kompose
--------------

Docker Compose ist ein leistungsfähiges Werkzeug, das die Orchestrierung von Multi-Container-Docker-Anwendungen vereinfacht. 

Anstatt mühsam einzelne Container manuell zu starten und zu konfigurieren, ermöglicht Docker Compose die Definition und Verwaltung von komplexen Anwendungen in einem einzigen YAML-Datei-Format.

- - - 
[kompose](https://kompose.io/) ist ein benutzerfreundliches und leistungsstarkes Werkzeug, das die Konvertierung von Docker Compose-Dateien in Kubernetes-Manifestdateien vereinfacht. Es wurde entwickelt, um Entwicklern den Übergang von Docker-basierten Entwicklungsumgebungen zu Kubernetes-Produktionsumgebungen zu erleichtern.

Zu Übungszwecken wandeln wir eine Docker Compose Datei nach Kubernetes.

Zuerst erstellen wir eine Docker Compose YAML-Datei:


In [ ]:
%%bash
cat <<%EOF% >docker-compose.yaml
services:
  customer:
    container_name: customer
    image: registry.gitlab.com/ch-mc-b/autoshop-ms/app/shop/customer:2.0.0
    ports:
      - "8080"      
  catalog:
    container_name: catalog
    image: registry.gitlab.com/ch-mc-b/autoshop-ms/app/shop/catalog:2.0.0
    ports:
      - "8080"       
  order:
    container_name: order
    image: registry.gitlab.com/ch-mc-b/autoshop-ms/app/shop/order:2.0.0
    ports:
      - "8080"            
  webshop:
    container_name: shop
    image: registry.gitlab.com/ch-mc-b/autoshop-ms/app/shop/shop:2.0.0
    ports:
      - "8080:8080"
%EOF%


Diese Datei wandeln wir mittels `kompose` nach Kubernetes um

In [ ]:
%%bash
rm -rf kompose
mkdir kompose
cd kompose && kompose -f ../docker-compose.yaml convert 
ls -l

Die mittels `kompose` erstellen Dateien übergeben wir an Kubernetes

In [ ]:
! kubectl create namespace kompose
! kubectl apply -f kompose --namespace kompose

Überprüfen die wrstellen Kubernetes Ressourcen und stellen fest, dass der Port von `webshop` nicht gegen aussen (LoadBalancer) geöffnet wurde.

In [ ]:
! kubectl get pods,services --namespace kompose

Dies korrigieren wir mittels eines kleinen Shellscripts und unsere Microservices sind nun ansprechbar

In [ ]:
! kubectl get service --namespace kompose -l io.kompose.service=webshop -o yaml | sed 's/ClusterIP/LoadBalancer/g' | kubectl apply -f -
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace kompose webshop -o=jsonpath='{ .spec.ports[0].nodePort }')

- - -
Aufräumen

In [ ]:
! kubectl delete namespace kompose